
# WRF–SUEWS 3-hour coupled demo

Idealized `em_quarter_ss` run with the SUEWS surface driver (`sf_surface_physics = 9`).

- Build: `compilation-20251120` (Homebrew GCC/GFortran, SUEWS linked via `wrf_suews.mk`).
- Runtime: 3 h integration, 10 min history output, single domain (42×42, 2 km).
- Case assets: `namelist.suews` + land/veg/urban tables from `SUEWS`; physics namelist at `../compilation-20251120/test/em_quarter_ss/namelist.input`.
- Output file analyzed below: `../compilation-20251120/test/em_quarter_ss/wrfout_d01_0001-01-01_00:00:00`.


In [ ]:

from pathlib import Path
import numpy as np
import xarray as xr

wrfout = Path("../compilation-20251120/test/em_quarter_ss/wrfout_d01_0001-01-01_00:00:00")
ds = xr.open_dataset(wrfout)

def to_hours(x):
    # Convert XTIME (cftime datetimes) to hours since start.
    base = x[0]
    return np.array([(t - base).total_seconds() / 3600 for t in x])

ds


In [ ]:

# Quick inventory of the SUEWS content and time axis
suews_vars = sorted([v for v in ds.data_vars if "SUEWS" in v])
time_hours = to_hours(ds["XTIME"].values)

print(f"time steps: {len(time_hours)} (0–{time_hours[-1]:.1f} h)")
print(f"first 3 time stamps: {time_hours[:3]} h ... last: {time_hours[-1]:.1f} h")
print(f"SUEWS variables: {len(suews_vars)} present; QF_SUEWS in file? {'QF_SUEWS' in ds}")
print("first 10 vars:")
for name in suews_vars[:10]:
    print("  -", name)


In [ ]:

# Fluxes at the domain center (time series)
import matplotlib.pyplot as plt

ci = ds.sizes["west_east"] // 2
cj = ds.sizes["south_north"] // 2
center = ds.isel(west_east=ci, south_north=cj)

flux_vars = {
    "KDOWN_SUEWS": "SW↓ (KDOWN)",
    "LDOWN_SUEWS": "LW↓ (LDOWN)",
    "QN_SUEWS": "Net all-wave (QN)",
    "AH_SUEWS": "Anthropogenic heat (AH)",
}

fig, ax = plt.subplots(figsize=(8, 4))
hours = to_hours(center["XTIME"].values)
for name, label in flux_vars.items():
    if name in center:
        ax.plot(hours, center[name], label=label)
ax.set_xlabel("Simulation time (h)")
ax.set_ylabel("Flux (W m$^{-2}$)")
ax.set_title("SUEWS surface fluxes @ domain center")
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()


In [ ]:

# Surface state evolution @ center
fig, axes = plt.subplots(1, 3, figsize=(13, 3), sharex=True)
hours = to_hours(center["XTIME"].values)
axes[0].plot(hours, center["TSURF_SUEWS"]); axes[0].set_ylabel("K"); axes[0].set_title("TSURF_SUEWS")
axes[1].plot(hours, center["SOILMOIST_SUEWS"]); axes[1].set_title("SOILMOIST_SUEWS")
axes[2].plot(hours, center["MELTWATERSTORE_SUEWS"]); axes[2].set_title("MELTWATERSTORE_SUEWS")
for ax in axes:
    ax.set_xlabel("Simulation time (h)")
    ax.grid(True, alpha=0.3)
fig.tight_layout()


In [ ]:

# Spatial footprint at the final time: TSURF_SUEWS and QN_SUEWS
fig, axes = plt.subplots(1, 2, figsize=(11, 4))
last = ds.isel(Time=-1)

pcm0 = axes[0].pcolormesh(last["TSURF_SUEWS"], shading="nearest", cmap="turbo")
axes[0].set_title("TSURF_SUEWS @ t=3 h")
fig.colorbar(pcm0, ax=axes[0], label="K")

pcm1 = axes[1].pcolormesh(last["QN_SUEWS"], shading="nearest", cmap="coolwarm")
axes[1].set_title("QN_SUEWS @ t=3 h")
fig.colorbar(pcm1, ax=axes[1], label="W m$^{-2}$")

for ax in axes:
    ax.set_xlabel("west_east index")
    ax.set_ylabel("south_north index")
fig.tight_layout()
